In [1]:
!pip install -q -r requirements.txt
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from datasets import load_dataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
import evaluate
import numpy as np
from torch.nn import functional as F
from typing import Union
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import pandas as pd
from collections import Counter
from dataset_class import *

import warnings
import sklearn.exceptions
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=sklearn.exceptions.UndefinedMetricWarning)

violet = '#702b9d'

You should consider upgrading via the '/share/apps/python/3.8.6/intel/bin/python -m pip install --upgrade pip' command.


/scratch/tl2546/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/scratch/tl2546/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/scratch/tl2546/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/scratch/tl2546/.local/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, ma

In [2]:
train_path = '/scratch/' + os.environ.get("USER", "") + '/data/jigsaw_training'
val_path =  '/scratch/' + os.environ.get("USER", "") + '/data/jigsaw_validation'
training_file_path = os.path.join(train_path, 'train.csv')
validation_file_path = os.path.join(val_path, 'validation_data.csv')

In [3]:
toxicity_train = pd.read_csv(f'{training_file_path}')
toxicity_train

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59848,0.000000,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,59849,0.000000,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,59852,0.000000,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,59855,0.000000,Is this something I'll be able to install on m...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1804869,6333967,0.000000,"Maybe the tax on ""things"" would be collected w...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,399385,approved,0,0,0,0,0,0.0,0,4
1804870,6333969,0.000000,What do you call people who STILL think the di...,0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,399528,approved,0,0,0,0,0,0.0,0,4
1804871,6333982,0.000000,"thank you ,,,right or wrong,,, i am following ...",0.000000,0.000000,0.000000,0.000000,0.0,NaN,NaN,...,399457,approved,0,0,0,0,0,0.0,0,4
1804872,6334009,0.621212,Anyone who is quoted as having the following e...,0.030303,0.030303,0.045455,0.621212,0.0,NaN,NaN,...,399519,approved,0,0,0,0,0,0.0,0,66


In [4]:
toxicity_train['label'] = np.where(toxicity_train.target > 0.5, 1, 0)

In [17]:
import numpy as np
import pandas as pd
import os
# import wordcloud

## TODO: complete dataset class just in case
class toxic_dataset():
    def __init__(self, df, text, toxicity, size = 10, label = None, seed = 1011):
        """
        Args:
            df (pd.DataFrame): original dataframe
            text (pd.Series): comment text
            toxicity (pd.Series): toxic score
            size: length of training set
            label: toxic (1) or not (0). Keep it in case to calculate accuracy, etc. make sure it is of the same length as text  
            seed: random seed. Default to 1011
        """
        self.df = df
        self.text = text
        self.toxicity = toxicity
        self.size = size
        self.label = label
        self.seed = seed
        self.rng = np.random.default_rng(seed = self.seed)
    def rate_toxicity(self, methods = None):
        """
        Args:
            methods: Defaults to None. Use the target 
            1. linear combination of target, severity toxicity, obscene, 
            identity_attach, insult, threat. see README.md for details.
            2. ?
        """

        pass
            
    def reset_all_indices(self):
        """
        Reset all the indices
        """
        # reset index
        self.text.reset_index(drop = True, inplace = True)
        self.toxicity.reset_index(drop = True, inplace = True)
        self.label.reset_index(drop = True, inplace = True)
        
    def down_sample(self, threshold, rate = 0.1):
        """
        downsample non-toxic texts
        """
        no_toxic = self.text[self.toxicity <= threshold]
        sample_ind = self.rng.integers(0, len(no_toxic), 
                                        int(len(no_toxic) * rate))
        no_toxic = no_toxic.iloc[no_toxic.index[sample_ind]]
        toxic = self.text[self.toxicity > threshold]
        self.text = pd.concat([no_toxic, toxic], axis = 0)
        self.rng.shuffle(self.text)

        # update other attributes
        self.toxicity = self.toxicity[self.text.index]
        self.label = self.label[self.text.index]
        self.reset_all_indices()
        
        
    def make_pairs(self, indices = np.zeros((1, 2))):
        ## TODO: need to tackle initial condition
        # terminate condition
        if len(indices) == self.size:
            self.indices = indices
            self.rearrange()
            self.reset_all_indices()
            print(self.text, self.toxicity, self.label)
            print('paired up ...')
            return

        add_ind = self.rng.integers(0, len(self.text), 
                                    size = (self.size - len(indices), 2))
        add_ind = np.unique(add_ind, axis = 0)
        return self.make_pairs(indices = np.vstack((indices, add_ind))) if len(indices) > 1 \
            else self.make_pairs(indices = add_ind)          
    
    
    def rearrange(self):
        indices = self.indices
        self.text = pd.concat([self.text.iloc[indices[:,0]].reset_index(drop= True), 
                                  self.text.iloc[indices[:,1]].reset_index(drop=True)], 
                              axis = 1)
        self.text.columns = ['more_toxic_text', 'less_toxic_text']
        
        self.toxicity = pd.concat([self.toxicity.loc[indices[:, 0]].reset_index(drop=True),
                               self.toxicity.loc[indices[:, 1]].reset_index(drop=True)],
                              axis=1)
        self.toxicity.columns = ['toxicity_more_toxic', 'toxicity_less_toxic']

        self.label = pd.concat([self.label.loc[indices[:, 0]].reset_index(drop=True),
                            self.label.loc[indices[:, 1]].reset_index(drop=True)],
                           axis=1)
        self.label.columns = ['labels_more_toxic', 'labels_less_toxic']



        swap = self.toxicity.iloc[:,0] <= self.toxicity.iloc[:,1]
        
        self.toxicity.loc[swap, ['toxicity_more_toxic', 'toxicity_less_toxic']] = \
            self.toxicity.loc[swap, ['toxicity_less_toxic', 'toxicity_more_toxic']].values
        
        self.text.loc[swap, ['more_toxic_text', 'less_toxic_text']] = \
            self.text.loc[swap, ['less_toxic_text', 'more_toxic_text']].values
        
        self.label.loc[swap, ['labels_more_toxic', 'labels_less_toxic']] = \
            self.label.loc[swap, ['labels_less_toxic', 'labels_more_toxic']].values
                
    def make_dataframe(self, down_sample = False, make_pairs = False):
        """
        Run after self.make_pairs
        """
        if down_sample:
            self.down_sample()
        if make_pairs:
            self.make_pairs()
        self.df = pd.concat([self.text, self.toxicity, self.label], axis = 1)
        print('made new dataframe ...')
        
        
    def graph(self):
        """
        placeholder for wordcloud
        """
        pass

In [18]:
toxic = toxic_dataset(df = toxicity_train, text = toxicity_train.comment_text, 
                      toxicity = toxicity_train.target, size = 1000, label = toxicity_train.label, seed = 1011)

In [19]:
toxic.make_pairs()

                                       more_toxic_text  \
0                           Or read your comments.....   
1    perhaps ms. cain can talk with  the other land...   
2    The point is: Imago Dei has doctrines that are...   
3    No taxes or messing with the Permanent Fund ne...   
4           It's hard to ignore a woman on her period.   
..                                                 ...   
995  It's 2017 and women and girls still think "we ...   
996  Hewing closely to the leftist tendency to prop...   
997  What Bavious stated here is so obvious that an...   
998               I luv people with 4 names. Sooo cool   
999  'despite a change in the indication he is faci...   

                                       less_toxic_text  
0    The RV sellers have insurance. The insurer has...  
1    If there's a list from which a judge can be ch...  
2    So many snowflakes upset over a civil question...  
3    Virtually all your questions regarding the Fir...  
4    Please, tell 

In [ ]:
# take a look at distribution
print(toxicity_train.target.describe()) ## unbalanced sample

fig, ax = plt.subplots(1,1, figsize = (8,4))
sns.set(style="whitegrid") 
sns.histplot(toxicity_train.target, binwidth = .05, color = violet, ax = ax)


In [ ]:
# take a look at distribution
print(toxicity_train.severe_toxicity.describe()) ## unbalanced sample

fig, ax = plt.subplots(1,1, figsize = (8,4))
sns.set(style="whitegrid") 
sns.histplot(toxicity_train.severe_toxicity, binwidth = .05, color = violet, ax = ax)


In [ ]:
train_jtc = pd.read_csv(f'{os.path.join(train_path, "train_jtc.csv")}')
train_jtc.nunique()

In [ ]:
# take a look at distribution
print(train_jtc.toxic.describe()) ## unbalanced sample

fig, ax = plt.subplots(1,1, figsize = (8,4))
sns.set(style="whitegrid") 
sns.histplot(train_jtc.toxic, binwidth = .05, color = violet, ax = ax)

In [ ]:
toxicity_valid = pd.read_csv(f'{validation_file_path}')
print(toxicity_valid.nunique())
print(toxicity_valid.shape)

In [ ]:
## preprocess validation dataset 
texts = set(toxicity_valid.less_toxic.to_list() + toxicity_valid.more_toxic.to_list())
text2id = {t:id for id,t in enumerate(texts)}
toxicity_valid['less_id'] = toxicity_valid['less_toxic'].map(text2id)
toxicity_valid['more_id'] = toxicity_valid['more_toxic'].map(text2id)
toxicity_valid_grouped = toxicity_valid.groupby(['less_id', 'more_id']).first().reset_index().drop(['less_id', 'more_id'], axis = 1)

In [ ]:
toxicity_valid_grouped